## Реализация LogitLens для мультимодальный нейронных сетей в фреймворке TransformerLens

In [1]:
!pip install datasets jaxtyping better_abc pillow wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 29.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

#### Связи проекта

In [2]:
import sys
from pathlib import Path

In [7]:
path = "."
# for kaggle
# path = "/kaggle/input/logitlens1-0/LogitLens/experiments/1.0-logit-lens/notebook"

ROOT = Path(path).absolute().parent.parent.parent

DATA = ROOT / "data"

In [8]:
if ROOT not in sys.path:
    sys.path.append(str(ROOT))

#### Импорт библиотек

In [10]:
import requests
import torch
from PIL import Image

import transformer_lens.utils as utils
from transformer_lens import HookedTransformer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1738401247.901508      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


#### Настройка логгирования

In [11]:
import logging

In [12]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)

logger = logging.getLogger(__name__)

logger.info("Logger has been configured successfully.")

#### Использование фреймфорка

In [13]:
torch.set_grad_enabled(False)

In [14]:
device = utils.get_device()
logger.info(f"Device: {device}")

In [15]:
model_name = "llava-hf/llava-1.5-7b-hf"

Создаем экземпляр класса HookedTransformer для llava

In [16]:
model = HookedTransformer.from_pretrained(model_name, device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.47s/it]
`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


Loaded pretrained model llava-hf/llava-1.5-7b-hf into HookedTransformer


Используем метод run_with_cache для llava

In [17]:
prompt = "USER: <image>\nWhat's the sign on the image? ASSISTANT:"

In [18]:
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
raw_image = Image.open(requests.get(url, stream=True).raw)

In [19]:
model_logits, model_cache = model.run_with_cache(prompt,
                                               input_images=raw_image,
                                               remove_batch_dim=False)

In [20]:
print(model_logits.shape)

torch.Size([1, 597, 32064])


Используем метод generate для llava

In [21]:
prompt = "USER: <image>\nWho is depicted in the picture? ASSISTANT:"

In [22]:
url = "https://thumbs.dreamstime.com/z/science-people-tablet-lab-advice-brainstorming-innovation-medicine-development-scientists-research-team-tech-facility-349449134.jpg"
raw_image = Image.open(requests.get(url, stream=True).raw)

In [24]:
print(model.generate(prompt, input_images=raw_image, max_new_tokens=30))

100%|██████████| 30/30 [01:03<00:00,  2.11s/it]


USER: <image> 
Who is depicted in the picture? ASSISTANT: In the picture, there is a group of doctors (medical professionals) wearing lab coats and goggles, discussing
